Make sure to select the GPU runtime in Runtime>Change Runtime Type.

In [1]:
!pip install datasets
!pip install accelerate -U
!pip install evaluate
#clear_output()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
from datasets import Dataset,load_dataset
import torch.cuda

Aggregated dataset

In [3]:
dataset0 = load_dataset('csv', data_files='aggregated_dataset.csv')
dataset = load_dataset('csv', data_files='aggregated_with_liar.csv')
dataset_liar = load_dataset('csv', data_files='liar_dataset.csv')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Tokenizer utilities

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"],  padding='max_length',truncation=True)


tokenized_datasets0 = dataset0.map(tokenize_function, batched=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets_liar = dataset_liar.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/9644 [00:00<?, ? examples/s]

Map:   0%|          | 0/9644 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Functions to compute metrics.

In [5]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
metric2 = evaluate.load("f1")

In [6]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)['accuracy']
    f1 = metric2.compute(predictions=predictions, references=labels)['f1']
    return {"accuracy": accuracy, "f1": f1}

### Cross Validation without liar


In [7]:
cv=5 #number of folds

In [8]:
ds_len=len(tokenized_datasets0['train'])

In [9]:
fold_size=ds_len//cv
fs=fold_size
print(fs)

1928


In [10]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

In [11]:
from transformers import set_seed
#from torch import manual_seed
#manual_seed(0)
import numpy as np
np.random.seed(0)
set_seed(42)

In [12]:
a_acc = []
b_acc = []
c_acc = []
liar_acc = []
total_acc = []

In [13]:
for i in range(cv):
    # Selects the first i folds and the last cv-i folds for training
    training_folds= list(range(0,fs*i)) + list(range(fs*(i+1),ds_len))
    train_dataset = tokenized_datasets0["train"].shuffle(seed=42).select( training_folds )
    # Selects the i-th fold for evaluation
    eval_fold = list(range(fs*i, fs*(i+1)))
    eval_dataset = tokenized_datasets0["train"].shuffle(seed=42).select( eval_fold )


    tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

    model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

    # Training parameters, see https://huggingface.co/docs/transformers/v4.35.2/main_classes/trainer#transformers.TrainingArguments
    # In particular, weight_decay to add?
    training_args = TrainingArguments(
        output_dir="test_trainer",
        num_train_epochs=2,
        evaluation_strategy="epoch",
        per_device_train_batch_size=14,
        per_device_eval_batch_size=14)

    # Trainer https://huggingface.co/docs/transformers/v4.35.2/main_classes/trainer#transformers.Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )

    trainer.train()

    A=eval_dataset.filter(lambda ex: ex['type']=='A')
    B=eval_dataset.filter(lambda ex: ex['type']=='B')
    C=eval_dataset.filter(lambda ex: ex['type']=='C')
    #D=eval_dataset.filter(lambda ex: ex['type']=='D')

    a_res=trainer.evaluate(A)["eval_accuracy"]
    b_res=trainer.evaluate(B)["eval_accuracy"]
    c_res=trainer.evaluate(C)["eval_accuracy"]
    #d_res=trainer.evaluate(D)["eval_accuracy"]

    a_acc.append(a_res)
    b_acc.append(b_res)
    c_acc.append(c_res)
    #d_acc.append(d_res)
    print(a_acc, b_acc, c_acc)

    tot_acc = (a_res*len(A)+b_res*len(B)+c_res*len(C)) / len(eval_dataset)
    print(tot_acc)
    total_acc.append( tot_acc)

    result_liar= trainer.evaluate(tokenized_datasets_liar['train'].shuffle(seed=42).select( range(400*i, 400*(i+1))))
    print(f'Eval accuracy on liar is {result_liar["eval_accuracy"]}')
    liar_acc.append(result_liar["eval_accuracy"])
    try:
        del model
        del trainer
        del training_args
        del tokenizer
    except:
        pass
    torch.cuda.empty_cache()

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.614500,0.528217,0.743776,0.739176
2,0.429200,0.494227,0.775934,0.779141


Filter:   0%|          | 0/1928 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1928 [00:00<?, ? examples/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.620100,0.542721,0.731328,0.746327
2,0.436700,0.512736,0.753631,0.753247


Filter:   0%|          | 0/1928 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1928 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1928 [00:00<?, ? examples/s]

[0.7370030581039755, 0.7147435897435898, 0.7020648967551623, 0.6977491961414791, 0.6609686609686609] [0.783210332103321, 0.7513711151736746, 0.7786811201445348, 0.7665198237885462, 0.757631822386679] [0.7852998065764023, 0.8045977011494253, 0.7966804979253111, 0.7966804979253111, 0.8104838709677419]
0.7536307053941909
Eval accuracy on liar is 0.58


In [14]:
def mean_sd(array):
  z=np.array(array)
  return (z.mean(), z.std())

In [16]:
print(mean_sd(a_acc))
print(mean_sd(b_acc))
print(mean_sd(c_acc))
print(mean_sd(liar_acc))
print(mean_sd(total_acc))

(0.7025058803425734, 0.024851470416517953)
(0.7674828427193512, 0.012085637022318551)
(0.7987484749088384, 0.008501208217630764)
(0.568, 0.03582596823534569)
(0.7644190871369294, 0.007744732968850681)


#Cross validation with liar (substitution)


In [ ]:
cv=5 #number of folds
ds_len=len(tokenized_datasets['train'])
fold_size=ds_len//cv
fs=fold_size
print(fs)
from transformers import set_seed
#from torch import manual_seed
#manual_seed(0)
import numpy as np
np.random.seed(0)
set_seed(42)
a_acc = []
b_acc = []
c_acc = []
d_acc = []
total_acc = []

1928


In [ ]:
for i in range(cv):
    # Selects the first i folds and the last cv-i folds for training
    training_folds= list(range(0,fs*i)) + list(range(fs*(i+1),ds_len))
    train_dataset = tokenized_datasets["train"].shuffle(seed=42).select( training_folds )
    # Selects the i-th fold for evaluation
    eval_fold = list(range(fs*i, fs*(i+1)))
    eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select( eval_fold )


    tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

    model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

    # Training parameters, see https://huggingface.co/docs/transformers/v4.35.2/main_classes/trainer#transformers.TrainingArguments
    # In particular, weight_decay to add?
    training_args = TrainingArguments(
        output_dir="test_trainer",
        num_train_epochs=2,
        evaluation_strategy="epoch",
        per_device_train_batch_size=14,
        per_device_eval_batch_size=14)

    # Trainer https://huggingface.co/docs/transformers/v4.35.2/main_classes/trainer#transformers.Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )

    trainer.train()

    A=eval_dataset.filter(lambda ex: ex['type']=='A')
    B=eval_dataset.filter(lambda ex: ex['type']=='B')
    C=eval_dataset.filter(lambda ex: ex['type']=='C')
    D=eval_dataset.filter(lambda ex: ex['type']=='D')

    a_res=trainer.evaluate(A)["eval_accuracy"]
    b_res=trainer.evaluate(B)["eval_accuracy"]
    c_res=trainer.evaluate(C)["eval_accuracy"]
    d_res=trainer.evaluate(D)["eval_accuracy"]

    a_acc.append(a_res)
    b_acc.append(b_res)
    c_acc.append(c_res)
    d_acc.append(d_res)
    print(a_acc, b_acc, c_acc, d_acc)

    tot_acc = (a_res*len(A)+b_res*len(B)+c_res*len(C) +d_res*len(D)) / len(eval_dataset)
    print(tot_acc)
    total_acc.append( tot_acc)
    try:
        del model
        del trainer
        del training_args
        del tokenizer
    except:
        pass
    torch.cuda.empty_cache()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.630300,0.547597,0.730809,0.735878
2,0.417900,0.568234,0.745332,0.743335


Filter:   0%|          | 0/1928 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1928 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1928 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/1928 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1928 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1928 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1928 [00:00<?, ? examples/s]

[0.7186544342507645, 0.6987179487179487, 0.6784660766961652, 0.7138263665594855, 0.6552706552706553] [0.7384393063583815, 0.7072135785007072, 0.746031746031746, 0.7358490566037735, 0.7781109445277361] [0.7872340425531915, 0.7911877394636015, 0.7883817427385892, 0.7842323651452282, 0.8266129032258065] [0.7244897959183674, 0.710594315245478, 0.7053140096618358, 0.72264631043257, 0.7415458937198067]
0.7603734439834025


In [ ]:
print(mean_sd(a_acc))
print(mean_sd(b_acc))
print(mean_sd(c_acc))
print(mean_sd(d_acc))
print(mean_sd(total_acc))

(0.6929870962990038, 0.023491785909623513)
(0.7411289264044688, 0.02270272763041957)
(0.7955297586252834, 0.015700695171652946)
(0.7209180649956115, 0.012580284287030943)
(0.7425311203319502, 0.010448895723563818)
